# Billboard hits in the year 2000: Exploratory data analysis

The data consists of 317 songs (all in the year 2000) and their evolution in the Billboard charts. For each song the data includes the artist, the length of the song, the genre, the date it entered the charts and the date of the peak in the charts. After this, each song has a timeline of their evolution in the charts. This timeline goes on for 76 weeks, which means that songs that have not been in the charts for 76 weeks has missing data in one of these columns. An interesting next step would be to find out for how many songs this is the case.

In [1]:
import pandas as pd
import numpy as np
billboard_df = pd.read_csv("../assets/billboard.csv")
billboard_df.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
billboard_df.count().head(15)

year               317
artist.inverted    317
track              317
time               317
genre              317
date.entered       317
date.peaked        317
x1st.week          317
x2nd.week          312
x3rd.week          307
x4th.week          300
x5th.week          292
x6th.week          280
x7th.week          269
x8th.week          260
dtype: int64

It is reassuring to see that the count does not give missing values for any of the track-specific information (such as artist, song length or genre) or the week of entrance in the data charts. However, what can also be seen from the data is that the number of columns is larger than it should be. No song in our sample has been in the charts for longer than 65 weeks, which is why we can reduce the number of columns. 

# Data cleaning

In [4]:
billboard_clean = billboard_df.iloc[:,1:72]
billboard_clean.rename(columns = {'artist.inverted':'artist'}, inplace = True)
billboard_clean = billboard_clean.rename(columns=lambda x: x.replace('x', ''))
billboard_clean = billboard_clean.rename(columns=lambda x: x.replace('week', ''))
billboard_clean = billboard_clean.rename(columns=lambda x: x.replace('th.', ''))
billboard_clean = billboard_clean.rename(columns=lambda x: x.replace('rd.', ''))
billboard_clean = billboard_clean.rename(columns=lambda x: x.replace('nd.', ''))
billboard_clean = billboard_clean.rename(columns=lambda x: x.replace('st.', ''))

A new dataframe is created out of the previous dataframe with raw data, where columns with only missing data have been trimmed. To make handling of the data easier, the column titles for the weekly timeline are changed to just the week number in the timeline. Artist.inverted is renamed to just artist, and the first column of the data (the year 2000) has been removed. 

All other missing data is not yet replaced. Observations with missing data should not be removed since it indicates that the song is no longer in the charts at a specific point, and intuitively it would not make sense to replace this data with a string or an integer.

Next up, I use the melt function in pandas to rearrange columns in the dataframe.

In [7]:
## The pandas melt function transforms the dataframe to a dataframe where every row indicates a weekly position in the chart, which means a song can have multiple entries.
billboard= pd.melt(billboard_clean, id_vars =("track","artist", "genre", "time", "date.entered", "date.peaked"),var_name= "Week", value_name="Ranking")
billboard['Week'].dtypes
billboard['Week']=billboard['Week'].astype(int)


billboard= billboard.dropna()
billboard['Ranking']=billboard['Ranking'].astype(int)
billboard["New_time"]='0:'+ billboard["time"]
billboard['timedelta'] = pd.to_timedelta(billboard["New_time"])
billboard['entered_to_datetime'] = pd.to_datetime(billboard['date.entered'])
billboard['peaked_to_datetime'] = pd.to_datetime(billboard['date.peaked'])

In [5]:
billboard.to_csv("billboard_clean.csv", sep=';')

# Data visualization

### For visualizations in Tableau, a new variable (weighted_ranking) was created, defined as:
##### weighted_ranking = 100 - Ranking

This assigns more weight to tracks that have been higher in the chart, and is thus a better proxy for popularity. 

<img src="Overall evolution of charts.png">

The visualizations above display an interesting story about the evolution of track popularity in the billboard charts in the year 2000. The top chart displays how long it takes for each song to leave the charts. The maximum number of weeks a song was in the charts, was 65 weeks. It is also interesting to see that there is a sharp decline in the amount of songs that are in the charts for longer than 20 weeks. 

The evolution of the median position in the charts shows a less clear trend. However, the initial downward slope suggests that songs generally take some time to reach their top position. After 10 weeks, the average position of songs that are still in the charts goes up. After 20 weeks there is a sharp drop in the median position of songs that are still in the charts. This is related to the findings in the previous graph, which suggests some sort of artificial cutoff after 20 weeks.

<img src="Genres.png">

As can be seen from the visualizations above, Rock was the most popular genre in the billboard charts in the year 2000. Rock songs in general stayed popular for a longer time as well. What is also interesting is that Rap songs (marked in pink) stay popular for a longer time than country songs.

<img src="Track popularity.png">

<img src="Artist popularity.png">